Extracted the key parts from https://github.com/JuliaMath/RandomMatrices.jl/blob/master/src/densities/TracyWidom.jl

In [ ]:
using SpecialFunctions, FastGaussQuadrature, LinearAlgebra

In [ ]:
airy_kernel(x, y) = x==y ? (airyaiprime(x))^2 - x * (airyai(x))^2 :
           (airyai(x) * airyaiprime(y) - airyai(y) * airyaiprime(x)) / (x - y)
ϕ(ξ, s) =  s + 10*tan(π*(ξ+1)/4) # Transformation from [-1,1] to (s,∞)
ϕ′(ξ) = (5π/2)*(sec(π*(ξ+1)/4))^2
K(ξ,η,s) = sqrt(ϕ′(ξ) * ϕ′(η)) * airy_kernel(ϕ(ξ,s), ϕ(η,s))

In [ ]:
# Tracy widom at s = -2 using "det"
n = 100 
s = -2

function K(s , n=100) 
    nodes,weights = gausslegendre(n)
    Symmetric( K.(nodes',nodes,s) .* (√).(weights) .* (√).(weights'))
end
det(I -  K(s))

In [ ]:
# Eigenvalues of airy Kernel at s = -2
λ = sort(eigvals(K(s)) , rev =true)
λ = λ[ λ .> 1e-15]

In [ ]:
prod(1 .- λ)  # product of 1-eigs = det = tracy widom

In [ ]:
nodes,weights = gausslegendre(7)

In [ ]:
n = 10
nodes,weights = gausslegendre(n)
hcat( [ weights ⋅ (x->x^k).(nodes) for k=0:2*n], [k%2==1 ? 0 : 2/(k+1) for k=0:2*n])

In [ ]:
n = 10
d = [0.0 for m =(n):-1:1.0]
e = [1/√(4-1/m^2) for m=1:(n-1)]
eigvals(SymTridiagonal(d,e))

In [ ]:
2*eigvecs(SymTridiagonal(d,e))[1,:].^2

In [ ]:
n= 1000
r = range(-1,stop=1,length=n)
V = r.^(0:n-1)'
Q, = qr(V)
T = Q'*diagm(r)*Q
T = SymTridiagonal(diag(T),abs.(diag(T,-1)))
round.(T.ev[1:10],digits=3)

In [ ]:
[1/√(4-1/m^2) for m=1:12]

## Lanczos on Airy Kernel
with reorthognalization, grabbed from Arbenz http://people.inf.ethz.ch/arbenz/ewp/Lnotes/chapter10.pdf

In [ ]:
function(lanczos(s,steps=10,n=50))
  A = copy(K(s,n));
  nodes,weights = gausslegendre(n)
  q = fill(1.0,n).*(√).(weights)
  q = normalize(q)
  Q = q
  r =  A * q
  α =  q ⋅ r
  r -= α * q
  β = norm(r)
  α_list,β_list = [α],[β]

  for j = 2:steps 
    v = q
    q = r / β  
    Q = [Q q]
    r = A * q - β*v
    α = q ⋅ r 
    r -= α * q 
    r -= Q*Q'r  # explicit reorthogonalization
    β = norm(r)  
    append!(α_list,α),append!(β_list,β)
  end
  T = SymTridiagonal(α_list, β_list[1:steps-1])
end

In [ ]:
s = -2.0
det(I-lanczos(s)), det(I-K(s))

In [ ]:
using Plots

In [ ]:
s = -5:.1:1
y1 = [det(I-lanczos(x,steps=1)) for x in s]
y2 = [det(I-lanczos(x,steps=2)) for x in s]
y3 = [det(I-lanczos(x,steps=3)) for x in s]
y4 = [det(I-lanczos(x,steps=4)) for x in s]
plot(s,[y1 y2 y3 y4])